In [144]:
#4 satellites Doppler 1 frequency

#Solving for pose and frequency offset

#Rotation of Earth and the effect of the ionosphere are included

#Doppler Method for Localizing a receiver
#Fausto Vega

using SatelliteDynamics
using LinearAlgebra
using PlotlyJS
using DelimitedFiles
using Distributions

In [145]:
# Declare simulation initial Epoch. Start time
epc0 = Epoch(2019, 1, 1, 12, 0, 0, 0.0)  #year, month, day, hour, minute, seconds, nanoseconds 

Epoch(2019-01-01T11:59:23.000Z)

In [146]:
# Declare initial state in terms of osculating orbital elements
#semi-major axis, eccentricity, inclination, RAAN, Argument of perigee, Mean anomaly

#1 and 2 on the same orbit, different initial mean anomaly

#iss1 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 185]; #converges at 189.0859
#iss2 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 205];
#iss3 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 190]; # converges a t 200.0859
#iss4 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 210]; # converges a t 200.0859

#approximately 500 km elevation from surface of Earth and around 100 km seperation at equator
#working good
#iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 190]; 
#iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 200];
#iss3 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 193]; 
#iss4 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 203]; 

#ARGOS Orbit ~850 km altitude

iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 190]; 
iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 200];
iss3 = [6871e3, 0.0004879, 90.6391, 196.5859, 151.2014, 193]; 
iss4 = [6871e3, 0.0004879, 90.6391, 196.5859, 151.2014, 203]; 



#Function to convert mean anomoaly to eccentric anomaly
E1 = anomaly_mean_to_eccentric(iss1[6], iss1[2], use_degrees=true); 
E2 = anomaly_mean_to_eccentric(iss2[6], iss2[2], use_degrees=true);
E3 = anomaly_mean_to_eccentric(iss3[6], iss3[2], use_degrees=true);
E4 = anomaly_mean_to_eccentric(iss4[6], iss4[2], use_degrees=true);

In [147]:
# Convert osculating elements to Cartesean state
# returns position and velocity (m, m/s)

eci0_1 = sOSCtoCART(iss1, use_degrees=true)
eci0_2 = sOSCtoCART(iss2, use_degrees=true)
eci0_3 = sOSCtoCART(iss3, use_degrees=true)
eci0_4 = sOSCtoCART(iss4, use_degrees=true)

6-element Vector{Float64}:
      -6.551891283502199e6
      -1.959561329014077e6
 -697065.2121487064
    -762.6662561944706
    -139.00657375176846
    7573.567651354772

In [148]:
#compute the satellite orbital period given the semi-major axis

#same for all 4 satellites since they have the same semi-major axis
T = orbit_period(iss1[1])

#final time for one orbit. Adds initial time to the orbital period

epcf = epc0 + T

Epoch(2019-01-01T13:33:51.144Z)

In [149]:
# Create an EarthInertialState orbit propagagator
#needs initial epoch of state and the state vector
orb1  = EarthInertialState(epc0, eci0_1, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb2  = EarthInertialState(epc0, eci0_2, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb3  = EarthInertialState(epc0, eci0_3, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb4  = EarthInertialState(epc0, eci0_4, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

# Propagate the orbit
# orbit until the final time

t_1, epc_1, eci_1 = sim!(orb1, epcf);
t_2, epc_2, eci_2 = sim!(orb2, epcf);
t_3, epc_3, eci_3 = sim!(orb3, epcf);
t_4, epc_4, eci_4 = sim!(orb4, epcf);

In [150]:
#equator position in cartesian coordinates
tag =[-6.128804e6, -1.590206e6, 776.1502e3]

#Examples of putting latitude and longitude and converting to ECEF coordinate frame
#Equitorial position
#tag_geof = [-165.4545, 16.98849, 0]

#North pole position
#tag_geof = [-165.4545, 46.98849, 0]

#tag = sGEOCtoECEF(tag_geof, use_degrees = true)

3-element Vector{Float64}:
     -6.128804e6
     -1.590206e6
 776150.2

In [151]:
#Equitorial Position
#In m
x0= tag[1] 
y0 = tag[2]
z0 = tag[3]

#λ0 = C_LIGHT/400e6 #nominal wavelength in m

f0 = 1e6

#r0 = [x0, y0, z0, λ0]
r0 = [x0, y0, z0, f0]

centroid_guess = [(eci_1[1,1]+eci_2[1,1]+eci_3[1,1]+eci_4[1,1])/4, (eci_1[2,1]+eci_2[2,1]+eci_3[2,1]+eci_4[2,1])/4, (eci_1[3,1]+eci_2[3,1]+eci_3[3,1]+eci_4[3,1])/4] 
onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
geodetic = [onearth[1], onearth[2], 0]

#Guess
xyz = sGEOCtoECEF(geodetic, use_degrees = true)

println(xyz)

[-5.99796596057736e6, -1.6894678391131323e6, -1.3604169754625652e6]


In [152]:
#Plot Satellite 1 and 2 Orbit
sat1 = scatter(x=eci_1[1,:], y=eci_1[2,:], z=eci_1[3,:], type="scatter3d", mode="lines", name="orbit 1&2")

#Plot Satellite 3 and 4 Orbit
sat3 = scatter(x=eci_3[1,:], y=eci_3[2,:], z=eci_3[3,:], type="scatter3d", mode="lines", name="orbit 3&4")

#Plot all 4 satellite initial positions
satellites = scatter(x=[eci_1[1,1],eci_2[1,1], eci_3[1,1], eci_4[1,1]], y=[eci_1[2,1],eci_2[2,1], eci_3[2,1], eci_4[2,1]],z=[eci_1[3,1],eci_2[3,1], eci_3[3,1], eci_4[3,1]], mode="markers", marker_size = 4, type="scatter3d", name="satellites")

#Plot Tag position
tag = scatter(x = [r0[1], r0[1]], y = [r0[2], r0[2]], z = [r0[3], r0[3]], type="scatter3d", name="tag", mode="markers", marker_size=5)

#Plot the Guess
guess = scatter(x = [xyz[1], xyz[1]], y = [xyz[2], xyz[2]], z = [xyz[3], xyz[3]], type="scatter3d", name="guess", mode="markers", marker_size=5)

#Plotting the sphere (Earth)
n = 100
u = range(-π, π; length = n)
v = range(0, π; length = n)
x = cos.(u) * sin.(v)'
y = sin.(u) * sin.(v)' 
z = ones(n) * cos.(v)' 

earth = surface(z=z*6371000, x=x*6371000, y=y*6371000)

#plot([sat1,sat3, satellites, tag, earth, guess]) #uncomment to see the plot

surface with fields type, x, y, and z


In [153]:
#Finding the Zenith Angle

function zenith_angle(satposes, r0)
    
    zenithangle = zeros(4)

    #vector between tag and the satellite
    normalvec = r0
    
    for i in 1:4
        
        vector = satposes[i,:] - r0[1:3]
        
        #Find the angle between the normal and the vector going from the tag to the satellte
        theta = acos(dot(normalvec,vector)/(norm(normalvec)*norm(vector))) * (180/pi)
        
        #save zenith angle
        zenithangle[i] = theta
    end
    
    return zenithangle
end

zenith_angle (generic function with 1 method)

In [154]:
#Testing zenith angle function
satposes = [eci_1[1,1] eci_1[2,1] eci_1[3,1];eci_2[1,1] eci_2[2,1] eci_2[3,1];eci_3[1,1] eci_3[2,1] eci_3[3,1];eci_4[1,1] eci_4[2,1] eci_4[3,1]]

zenith_angles = zenith_angle(satposes, r0[1:3])

println(zenith_angles)

#check all angles to see if it is on the horizon 
inhorizon = all(x->x<80, zenith_angles)

[93.63237107183178, 82.84377330309063, 91.00377335928297, 78.31523678325279]


false

In [155]:
#Obtain the orbital elements after each time step (not used in the sim) 
dim = size(t_1)[1]

all_elements = zeros(6,dim)
for i in 1:dim
    
    orbital_elements = sCARTtoOSC(eci_1[:,1], use_degrees=true)
    all_elements[:,i] = orbital_elements
end 

In [156]:
#velocity is the last 3 elements of each column in eci matrices
eci_1[:,1]

6-element Vector{Float64}:
    -6.291285386249863e6
    -1.66264810311861e6
    -2.2161566541800327e6
 -2394.9708335284654
  -540.1532693380549
  7206.151917789134

In [157]:
#Generate TEC Distribution
mu = 8e-4
sigma = 3e-4
lb = 3e-4
ub = 13e-4
d = Truncated(Normal(mu,sigma), lb, ub)

#f = 1575.42e6 # in 1/s L1 frequency
f = 400e6 # in 1/s system frequency
h = 350e3 #in meters

350000.0

In [158]:
using ForwardDiff
using BlockDiagonals

#scale used when estimating f0 (transmitting frequency)

# distance_scale = R_EARTH*(1e-3)*1e-4 # scales km to custom scale working

# time_scale = 1/(C_LIGHT/R_EARTH) #scales seconds to custom scale

# velocity_scale = distance_scale/time_scale #scales velocities to custom scale

# frequency_scale = 1e5 #working for f0 = 400e6

# c = 10000 #km/s

# # working to estimate bdot 
distance_scale = R_EARTH*(1e-3) # scales km to custom scale working

time_scale = (1/(C_LIGHT/R_EARTH))*1e4 #scales seconds to custom scale

velocity_scale = distance_scale/time_scale #scales velocities to custom scales

frequency_scale = 1

c = 1*1e4 #km/s

#udpated scale to estimate a frequency offset bdot


10000.0

In [159]:
1/distance_scale

0.0001567856114959475

In [160]:
velocity_scale

29.9792458

In [161]:
#satposet is all the satellite known positions and the measured position
#x is what we want to solve for
#satpose is the satellite position and velocities as well as the measured change in frequency

function doppler_residual(x, sat_pose_f, time, tag_vel)
    
    #true tag velocity
    #tag_vel = [10, 12, 7]*1e-3/velocity_scale
    
    #println("this is tag_vel: ", tag_vel)
    #the measured tag velocity will have noise
    
    omega = OMEGA_EARTH*time_scale #transform to custom scale
    
    #wavelength = 0.7494*1e-3/distance_scale #for a nominal frequency of 400 MHz
    
    omegahat = [0 -omega 0; omega 0 0; 0 0 0]
    
    A1 = [cos(omega*time[1]) sin(omega*time[1]) 0;
          -sin(omega*time[1]) cos(omega*time[1]) 0;
          0 0 1];
    A2 = [cos(omega*time[2]) sin(omega*time[2]) 0;
          -sin(omega*time[2]) cos(omega*time[2]) 0;
          0 0 1];
    A3 = [cos(omega*time[3]) sin(omega*time[3]) 0;
          -sin(omega*time[3]) cos(omega*time[3]) 0;
          0 0 1];
    A4 = [cos(omega*time[4]) sin(omega*time[4]) 0;
          -sin(omega*time[4]) cos(omega*time[4]) 0;
          0 0 1];
    
    #residuals used to estimate transmitting frequency as an unknown
    #res1 = (x[4]/c)*0.5*(1/norm(x[1:3] - A1*sat_pose_f[1,1:3]))*(-2*x[1:3]'*A1*omegahat*sat_pose_f[1,1:3] - 2*x[1:3]'*A1*sat_pose_f[1,4:6]+sat_pose_f[1,1:3]'*sat_pose_f[1,4:6]+sat_pose_f[1,4:6]'*sat_pose_f[1,1:3])- sat_pose_f[1,7]
    #res2 = (x[4]/c)*0.5*(1/norm(x[1:3] - A2*sat_pose_f[2,1:3]))*(-2*x[1:3]'*A2*omegahat*sat_pose_f[2,1:3] - 2*x[1:3]'*A2*sat_pose_f[2,4:6]+sat_pose_f[2,1:3]'*sat_pose_f[2,4:6]+sat_pose_f[2,4:6]'*sat_pose_f[2,1:3])- sat_pose_f[2,7]
    #res3 = (x[4]/c)*0.5*(1/norm(x[1:3] - A3*sat_pose_f[3,1:3]))*(-2*x[1:3]'*A3*omegahat*sat_pose_f[3,1:3] - 2*x[1:3]'*A3*sat_pose_f[3,4:6]+sat_pose_f[3,1:3]'*sat_pose_f[3,4:6]+sat_pose_f[3,4:6]'*sat_pose_f[3,1:3])- sat_pose_f[3,7]
    #res4 = (x[4]/c)*0.5*(1/norm(x[1:3] - A4*sat_pose_f[4,1:3]))*(-2*x[1:3]'*A4*omegahat*sat_pose_f[4,1:3] - 2*x[1:3]'*A4*sat_pose_f[4,4:6]+sat_pose_f[4,1:3]'*sat_pose_f[4,4:6]+sat_pose_f[4,4:6]'*sat_pose_f[4,1:3])- sat_pose_f[4,7]
    
    #extra_term = tag_vel'*x[1:3] + x[1:3]'*tag_vel - 2*tag_vel'*A1*sat_pose_f[1,1:3]
    
    #println("this is extra term: ", extra_term)
    
    #velocity of tag included in residual 
    #residuals to estimate a frequency offset bdot
    res1 = (f*frequency_scale/c)*(0.5*(1/norm(x[1:3] - A1*sat_pose_f[1,1:3]))*(tag_vel'*x[1:3] + x[1:3]'*tag_vel - 2*tag_vel'*A1*sat_pose_f[1,1:3]-2*x[1:3]'*A1*omegahat*sat_pose_f[1,1:3] - 2*x[1:3]'*A1*sat_pose_f[1,4:6]+sat_pose_f[1,1:3]'*sat_pose_f[1,4:6]+sat_pose_f[1,4:6]'*sat_pose_f[1,1:3]) + x[4]) - sat_pose_f[1,7]
    res2 = (f*frequency_scale/c)*(0.5*(1/norm(x[1:3] - A2*sat_pose_f[2,1:3]))*(tag_vel'*x[1:3] + x[1:3]'*tag_vel - 2*tag_vel'*A2*sat_pose_f[2,1:3]-2*x[1:3]'*A2*omegahat*sat_pose_f[2,1:3] - 2*x[1:3]'*A2*sat_pose_f[2,4:6]+sat_pose_f[2,1:3]'*sat_pose_f[2,4:6]+sat_pose_f[2,4:6]'*sat_pose_f[2,1:3]) + x[4]) - sat_pose_f[2,7]
    res3 = (f*frequency_scale/c)*(0.5*(1/norm(x[1:3] - A3*sat_pose_f[3,1:3]))*(tag_vel'*x[1:3] + x[1:3]'*tag_vel - 2*tag_vel'*A3*sat_pose_f[3,1:3]-2*x[1:3]'*A3*omegahat*sat_pose_f[3,1:3] - 2*x[1:3]'*A3*sat_pose_f[3,4:6]+sat_pose_f[3,1:3]'*sat_pose_f[3,4:6]+sat_pose_f[3,4:6]'*sat_pose_f[3,1:3]) + x[4]) - sat_pose_f[3,7]
    res4 = (f*frequency_scale/c)*(0.5*(1/norm(x[1:3] - A4*sat_pose_f[4,1:3]))*(tag_vel'*x[1:3] + x[1:3]'*tag_vel - 2*tag_vel'*A4*sat_pose_f[4,1:3]-2*x[1:3]'*A4*omegahat*sat_pose_f[4,1:3] - 2*x[1:3]'*A4*sat_pose_f[4,4:6]+sat_pose_f[4,1:3]'*sat_pose_f[4,4:6]+sat_pose_f[4,4:6]'*sat_pose_f[4,1:3]) + x[4]) - sat_pose_f[4,7]
    
    return [res1; res2; res3; res4]
   
end

doppler_residual (generic function with 1 method)

In [162]:
#pose1-4 are satellite locations and velocities
#r0 = [xyz of tag, emmitted frequency]

#gets the true measurment
function measurment(r0, pose1, pose2, pose3, pose4, time)
    
    #Make sure to scale all positions
    
    omega = OMEGA_EARTH*time_scale #change to custom time scale
    
    #wavelength = 0.7494*1e-3/distance_scale #for a nominal frequency of 400 MHz
    
    omegahat = [0 -omega 0; omega 0 0; 0 0 0]
    
    A1 = [cos(omega*time[1]) sin(omega*time[1]) 0;
          -sin(omega*time[1]) cos(omega*time[1]) 0;
          0 0 1];
    A2 = [cos(omega*time[2]) sin(omega*time[2]) 0;
          -sin(omega*time[2]) cos(omega*time[2]) 0;
          0 0 1];
    A3 = [cos(omega*time[3]) sin(omega*time[3]) 0;
          -sin(omega*time[3]) cos(omega*time[3]) 0;
          0 0 1];
    A4 = [cos(omega*time[4]) sin(omega*time[4]) 0;
          -sin(omega*time[4]) cos(omega*time[4]) 0;
          0 0 1];
    
    #measurments for estimating a transmitting frequency
    #deltaf1 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A1*pose1[1:3]))*(-2*r0[1:3]'*A1*omegahat*pose1[1:3] - 2*r0[1:3]'*A1*pose1[4:6]+pose1[1:3]'*pose1[4:6]+pose1[4:6]'*pose1[1:3])
    #deltaf2 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A2*pose2[1:3]))*(-2*r0[1:3]'*A2*omegahat*pose2[1:3] - 2*r0[1:3]'*A2*pose2[4:6]+pose2[1:3]'*pose2[4:6]+pose2[4:6]'*pose2[1:3])
    #deltaf3 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A3*pose3[1:3]))*(-2*r0[1:3]'*A3*omegahat*pose3[1:3] - 2*r0[1:3]'*A3*pose3[4:6]+pose3[1:3]'*pose3[4:6]+pose3[4:6]'*pose3[1:3])
    #deltaf4 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A4*pose4[1:3]))*(-2*r0[1:3]'*A4*omegahat*pose4[1:3] - 2*r0[1:3]'*A4*pose4[4:6]+pose4[1:3]'*pose4[4:6]+pose4[4:6]'*pose4[1:3])
    
    #velocity not in the measurment
    #measurments to estimate out a bdot value (frequency offset)
    deltaf1 = (f*frequency_scale/c)*(0.5*(1/norm(r0[1:3] - A1*pose1[1:3]))*(-2*r0[1:3]'*A1*omegahat*pose1[1:3] - 2*r0[1:3]'*A1*pose1[4:6]+pose1[1:3]'*pose1[4:6]+pose1[4:6]'*pose1[1:3]) + r0[4])
    deltaf2 = (f*frequency_scale/c)*(0.5*(1/norm(r0[1:3] - A2*pose2[1:3]))*(-2*r0[1:3]'*A2*omegahat*pose2[1:3] - 2*r0[1:3]'*A2*pose2[4:6]+pose2[1:3]'*pose2[4:6]+pose2[4:6]'*pose2[1:3]) + r0[4])
    deltaf3 = (f*frequency_scale/c)*(0.5*(1/norm(r0[1:3] - A3*pose3[1:3]))*(-2*r0[1:3]'*A3*omegahat*pose3[1:3] - 2*r0[1:3]'*A3*pose3[4:6]+pose3[1:3]'*pose3[4:6]+pose3[4:6]'*pose3[1:3]) + r0[4])
    deltaf4 = (f*frequency_scale/c)*(0.5*(1/norm(r0[1:3] - A4*pose4[1:3]))*(-2*r0[1:3]'*A4*omegahat*pose4[1:3] - 2*r0[1:3]'*A4*pose4[4:6]+pose4[1:3]'*pose4[4:6]+pose4[4:6]'*pose4[1:3]) + r0[4])
    
    return [deltaf1, deltaf2, deltaf3, deltaf4]
end

measurment (generic function with 1 method)

In [163]:
#Equitorial Position

#In km
x0= x0*1e-3 #x tag position
y0 = y0*1e-3 #y tag position
z0 = z0*1e-3 #z tag position
#f0 = 1e6 #nominal frequency (works)
#f0 = 400e6 #working

bdot = 1e4*1e-3 # working

#r0_scaled = [x0/distance_scale, y0/distance_scale, z0/distance_scale, f0/frequency_scale]

r0_scaled = [x0/distance_scale, y0/distance_scale, z0/distance_scale, bdot/velocity_scale]

4-element Vector{Float64}:
 -0.9609082828788089
 -0.24932142011452468
  0.12168918371970193
  0.33356409519815206

In [164]:
diff = 25e3*1e-3/distance_scale 

0.003919640287398688

In [165]:
function receiver_pose(all_sats_scaled, guess, zenith_angles, time, zdot) # remember to scale the guess to custom units

    n = 1000 # number of iterations
    #all_r0 = NaN*[zeros(4) for i = 1:n]
    all_r0 = [zeros(3) for i = 1:n] #obtain all the tag positions
    
    sat1poses = zeros(7,n)
    sat2poses = zeros(7,n)
    sat3poses = zeros(7,n)
    sat4poses = zeros(7,n)

    all_sats_noise = zeros(4,4)

    iters = 0

     #Monte Carlo Simulation
    for i in 1:n
        #Parameters for line search
        #b = 0.01
        b = 0.1
        c_=0.5
        #β = 10
        β = 1.0
        α = 1
        
        #create noise from normal distribution
        
        #scaled to the variable (0.1m error for distance & 1e-11 for time)
        gpsnoise = randn(12)*(0.1*1e-3/distance_scale) #0.1 m switch to km then to custom scale
        velocitynoise = randn(12)*(0.01*1e-3/velocity_scale)# 0.01 m/s
        doppler_noise = randn(12)* 1/frequency_scale #0.001 ~ 500's of error
        
        #doppler_noise = randn(12)* 0/frequency_scale #0.001 ~ 500's of error
        
        TEC = rand(d,4) #vTEC for each satellite from custom distribution
        
        #tag_vel_noise = randn(3)*(1*1e-3/velocity_scale)# 1 m/s noise
        
        tag_vel_noise = zeros(3) #zero noise
    
        tag_vel = [2, 5, 3]*1e-3/velocity_scale
        
        tag_vel_w_noise = tag_vel + tag_vel_noise
        
        #Calculating random TEC time delay
        #for i=1:4
        
        #    OF[i] = (1-((R_EARTH*sind(zenith_angles[i]))/(R_EARTH+hi))^2)^(-1/2) #normal units (m and s)
        #    Ip[i] = ((40.3*TEC[i])/(f^2))*OF[i] * 1e-3 #scale to km to use the custom unit
        #    Ip_scaled[i] = Ip[i]/distance_scale #scale to custom units
        
        #end
        
        Idot1 = (((40.3*TEC[1])*((R_EARTH*sind(zenith_angles[1]))*(R_EARTH*cosd(zenith_angles[1])*zdot[1])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[1]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        Idot2 = (((40.3*TEC[2])*((R_EARTH*sind(zenith_angles[2]))*(R_EARTH*cosd(zenith_angles[2])*zdot[2])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[2]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        Idot3 = (((40.3*TEC[3])*((R_EARTH*sind(zenith_angles[3]))*(R_EARTH*cosd(zenith_angles[3])*zdot[3])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[3]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        Idot4 = (((40.3*TEC[4])*((R_EARTH*sind(zenith_angles[4]))*(R_EARTH*cosd(zenith_angles[4])*zdot[4])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[4]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        
        iono1 = (f/c)*Idot1
        iono2 = (f/c)*Idot2
        iono3 = (f/c)*Idot3
        iono4 = (f/c)*Idot4
        
        scaled_iono1 = iono1/distance_scale*velocity_scale
        
        #println("this is scaled_iono1: ", scaled_iono1) #in Hz
        
        sat1_noise = [all_sats_scaled[1,1]+gpsnoise[1],all_sats_scaled[1,2] + gpsnoise[2],all_sats_scaled[1,3]+gpsnoise[3], all_sats_scaled[1,4]+velocitynoise[1], all_sats_scaled[1,5]+velocitynoise[2],all_sats_scaled[1,6]+velocitynoise[3], all_sats_scaled[1,7]+iono1+doppler_noise[1]]
        sat2_noise =[all_sats_scaled[2,1]+gpsnoise[4],all_sats_scaled[2,2] + gpsnoise[5],all_sats_scaled[2,3]+gpsnoise[6], all_sats_scaled[2,4]+velocitynoise[4], all_sats_scaled[2,5]+velocitynoise[5],all_sats_scaled[2,6]+velocitynoise[6], all_sats_scaled[2,7]+iono2+doppler_noise[2]]
        sat3_noise = [all_sats_scaled[3,1]+gpsnoise[7],all_sats_scaled[3,2] + gpsnoise[8],all_sats_scaled[3,3]+gpsnoise[9], all_sats_scaled[3,4]+velocitynoise[7], all_sats_scaled[3,5]+velocitynoise[8],all_sats_scaled[3,6]+velocitynoise[9], all_sats_scaled[3,7]+iono3+doppler_noise[3]]
        sat4_noise = [all_sats_scaled[4,1]+gpsnoise[10],all_sats_scaled[4,2] + gpsnoise[11],all_sats_scaled[4,3]+gpsnoise[12], all_sats_scaled[4,4]+velocitynoise[10], all_sats_scaled[4,5]+velocitynoise[11],all_sats_scaled[4,6]+velocitynoise[12], all_sats_scaled[4,7]+iono4+doppler_noise[4]]

        all_sats_noise = vcat(sat1_noise',sat2_noise',sat3_noise',sat4_noise')
        
        sat1poses[:,i] = sat1_noise
        sat2poses[:,i] = sat2_noise
        sat3poses[:,i] = sat3_noise
        sat4poses[:,i] = sat4_noise
        
        X = NaN*[zeros(4) for i = 1:n]
        R = NaN*[zeros(4) for i = 1:n]
        
        #Centroid guess. Working for estimating transmit frequency
        #X[1] = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 410e6/frequency_scale]
        
        
        #working
        X[1] = [r0_scaled[1]+diff, r0_scaled[2]+diff, r0_scaled[3]+diff, 1.002*1e4*1e-3/velocity_scale]

        
        #New guess to estimate bdot
        #X[1] = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 1.002*1e4*1e-3/velocity_scale]

        #X[1] = r0_scaled
        
        #println("this is initial guess: ", X[1])
        #Initial residual
        R[1] = doppler_residual(X[1], all_sats_noise, time, tag_vel_w_noise)

        iters = 0

        for k=1:1000
            
            R[k] = doppler_residual(X[k], all_sats_noise, time, tag_vel_w_noise) #calculate residual
            
            iters += 1

            #if(norm(R[k]) < 1e-6)
            #if(norm(R[k]) < 1e-8)
            if(norm(R[k]) < 1e-10)
                
                break

            end

            jacobian = ForwardDiff.jacobian(dx -> doppler_residual(dx, all_sats_noise, time, tag_vel_w_noise), X[k])
            
            if(rank(jacobian) != 4) #if not full rank, move on to next iteration
                
                break

            end
            
            conditionnum = cond(jacobian)
            
            #println("this is residual: ", norm(R[k]))
            
            #println("this is the condition number: ", conditionnum)
            
            #println("this is the jacobian: ", jacobian)
            
            #println("this is X[k]: ", X[k])

            deltax = (jacobian)\-R[k]
            
            if iters > 50
                break
            end
            
            while norm(doppler_residual(X[k] + α*deltax, all_sats_noise, time, tag_vel_w_noise)) > norm(doppler_residual(X[k], all_sats_noise, time, tag_vel_w_noise) + b*α*jacobian'*deltax)

                α = c_*α
                #print("this is alpha: ", α)
            end

            X[k+1] = X[k] + α*deltax
             
        end  
        
        R = R[1:iters]
        X = X[1:iters]

        all_r0[i] = X[end]

    end
    x_values = zeros(n)
    y_values = zeros(n)
    z_values = zeros(n)

    for j in 1:n

        x_values[j] = all_r0[j][1]
        y_values[j] = all_r0[j][2]
        z_values[j] = all_r0[j][3]

    end
    
    #mean calculation
    mean = sum(all_r0, dims = 1)/ n
    mean = mean[1]
    #for estimating frequency
    #mean_rescaled = [mean[1:3]*distance_scale*1e3; mean[4]*frequency_scale] # in meters
    
    #For estimating bdot
    mean_rescaled = [mean[1:3]*distance_scale*1e3; mean[4]*1e3*velocity_scale] # in meters

    return mean_rescaled, all_r0, iters
end

receiver_pose (generic function with 1 method)

In [166]:
#Navigation
#count = 0
n=1000
PDOP_array = zeros(size(eci_1)[2],1)
for i=1:size(eci_1)[2] - 1
    
    satposes = [eci_1[1,i] eci_1[2,i] eci_1[3,i];eci_2[1,i] eci_2[2,i] eci_2[3,i];eci_3[1,i] eci_3[2,i] eci_3[3,i];eci_4[1,i] eci_4[2,i] eci_4[3,i]]
    satposes2 = [eci_1[1,i+1] eci_1[2,i+1] eci_1[3,i+1];eci_2[1,i+1] eci_2[2,i+1] eci_2[3,i+1];eci_3[1,i+1] eci_3[2,i+1] eci_3[3,i+1];eci_4[1,i+1] eci_4[2,i+1] eci_4[3,i+1]]
    zenith_angles = zenith_angle(satposes, r0[1:3])
    zenith_angles2 = zenith_angle(satposes2, r0[1:3])
    
    zdot = zenith_angles - zenith_angles2

    #check all angles to see if it is on the horizon 
    
    inhorizon = all(x->x<70, zenith_angles)
    
    if inhorizon==false
        
        continue #skip the current iteration
    end

    #In km then to custom units
    r1 = [eci_1[1,i], eci_1[2,i], eci_1[3,i]]*1e-3/distance_scale
    r2 = [eci_2[1,i], eci_2[2,i], eci_2[3,i]]*1e-3/distance_scale
    r3 = [eci_3[1,i], eci_3[2,i], eci_3[3,i]]*1e-3/distance_scale
    r4 = [eci_4[1,i], eci_4[2,i], eci_4[3,i]]*1e-3/distance_scale
    
    v1 = [eci_1[4,i], eci_1[5,i], eci_1[6,i]]*1e-3/velocity_scale
    v2 = [eci_2[4,i], eci_2[5,i], eci_2[6,i]]*1e-3/velocity_scale
    v3 = [eci_3[4,i], eci_3[5,i], eci_3[6,i]]*1e-3/velocity_scale
    v4 = [eci_4[4,i], eci_4[5,i], eci_4[6,i]]*1e-3/velocity_scale

    pose1 = [r1' v1']
    pose2 = [r2' v2']
    pose3 = [r3' v3']
    pose4 = [r4' v4']
    
    time = [0.006, 0.006, 0.006, 0.008]/time_scale # assume a fixed time for the rotation matrix
    
    #calculate the rdots. need to build the pose matrices
    deltaf = measurment(r0_scaled, pose1, pose2, pose3, pose4, time)
    
    pose1_rdot1 = [pose1 deltaf[1]]
    pose2_rdot2 = [pose2 deltaf[2]]
    pose3_rdot3 = [pose3 deltaf[3]]
    pose4_rdot4 = [pose4 deltaf[4]]
    
    all_sats_scaled = vcat(pose1_rdot1,pose2_rdot2,pose3_rdot3,pose4_rdot4)
    
    #Generate a guess at the centroid of all 4 sats on the surface of the Earth. Scaled to km
    
    centroid_guess = [(eci_1[1,i]+eci_2[1,i]+eci_3[1,i]+eci_4[1,i])/4, (eci_1[2,i]+eci_2[2,i]+eci_3[2,i]+eci_4[2,i])/4, (eci_1[3,i]+eci_2[3,i]+eci_3[3,i]+eci_4[3,i])/4] 
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    xyz_guess = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3 #change to km

    xyz_guess
    
    #Find all possible positions from montecarlo and send unscaled guess
    mean_rescaled, all_r0, iters = receiver_pose(all_sats_scaled, xyz_guess, zenith_angles, time, zdot)
    
    all_r0_scaled = zeros(n,4)

    #Rescale back to units (m and s)
    for i in 1:n
    
        #all_r0_scaled[i,:] = [all_r0[i][1:3]*distance_scale*1e3; all_r0[i][4]*frequency_scale]
        all_r0_scaled[i,:] = [all_r0[i][1:3]*distance_scale*1e3; all_r0[i][4]*1e3*velocity_scale]

    end
    
    totalsum = zeros(4,4)

    for i in 1:n
        value = all_r0_scaled[i,:] - mean_rescaled
        matrixvalue = value*transpose(value)
        totalsum += matrixvalue
    
    end
    
    covariancematrix = totalsum/n
    
    PDOP = sqrt(covariancematrix[1,1] + covariancematrix[2,2] + covariancematrix[3,3])
    PDOP_array[i,1] = PDOP
    println("cov position erors: ", PDOP)
    println("Mean (m): ", mean_rescaled)
    println("Iterations: ", iters)
    println("Timestep: ", i)

end    

cov position erors: 5571.278476357755
Mean (m): [-6.125045101305739e6, -1.592341579823245e6, 775257.3448537396, 9998.217116387803]
Iterations: 5
Timestep: 261
cov position erors: 5666.491044774419
Mean (m): [-6.125041457048515e6, -1.5923428972501573e6, 775185.9434861091, 9997.759055755521]
Iterations: 5
Timestep: 262
cov position erors: 5942.2193486326505
Mean (m): [-6.124735217385257e6, -1.5926800858262433e6, 775025.881371748, 9997.390034035465]
Iterations: 5
Timestep: 263
cov position erors: 6147.033940233979
Mean (m): [-6.12454782968786e6, -1.5929054028735412e6, 774870.486165494, 9996.682083986283]
Iterations: 5
Timestep: 264
cov position erors: 6073.814496500452
Mean (m): [-6.124355444804049e6, -1.593218147619465e6, 774744.385778736, 9996.39787249449]
Iterations: 5
Timestep: 265
cov position erors: 6362.082845993952
Mean (m): [-6.124204183704314e6, -1.5933678654534083e6, 774593.2179231215, 9995.782769858833]
Iterations: 5
Timestep: 266
cov position erors: 6558.7393814145025
Mean (m

cov position erors: 12181.518625557777
Mean (m): [-6.131835261057015e6, -1.5764615487598823e6, 782756.937402597, 10019.964077380062]
Iterations: 5
Timestep: 313
cov position erors: 10856.129366464113
Mean (m): [-6.1315540547774285e6, -1.5767527090690637e6, 782532.5242733625, 10019.202762761068]
Iterations: 5
Timestep: 314
cov position erors: 11158.841685267418
Mean (m): [-6.131158584220848e6, -1.5776409639976798e6, 781991.9625139115, 10017.437974041217]
Iterations: 5
Timestep: 315
cov position erors: 10099.455196804838
Mean (m): [-6.130783334695163e6, -1.5783260886279098e6, 781598.7169517215, 10016.091191271706]
Iterations: 5
Timestep: 316
cov position erors: 10156.770425228662
Mean (m): [-6.130360564134961e6, -1.5793756438462131e6, 781031.5331491925, 10014.131120552482]
Iterations: 5
Timestep: 317
cov position erors: 9428.641149323745
Mean (m): [-6.1301707212893935e6, -1.5796574886651628e6, 780813.8658158589, 10013.45268993875]
Iterations: 5
Timestep: 318
cov position erors: 9109.8757

In [167]:
final_PDOP = PDOP_array[PDOP_array .!=0]

PDOP_max = findmax(final_PDOP)[1]
PDOP_min = findmin(final_PDOP)[1]

println("Maximum PDOP: ", PDOP_max)
println("Minimum PDOP: ", PDOP_min)

Maximum PDOP: 159622.35501685223
Minimum PDOP: 4820.299672036124


In [25]:
r0_scaled

4-element Vector{Float64}:
 -0.9609082828788089
 -0.24932142011452468
  0.12168918371970193
  0.33356409519815206

In [26]:
initial_guess = [-9625.008623511323, -2580.926606494148, 835.4931473148392, 3.9e6]

4-element Vector{Float64}:
 -9625.008623511323
 -2580.926606494148
   835.4931473148392
     3.9e6

In [ ]:
r0_scaled - initial_guess

In [ ]:
actualpose =[-6.128804e6, -1.590206e6, 776.1502e3, 1e6]

In [ ]:
log10(79.34904895113688)

In [ ]:
i=303

satposes = [eci_1[1,i] eci_1[2,i] eci_1[3,i];eci_2[1,i] eci_2[2,i] eci_2[3,i];eci_3[1,i] eci_3[2,i] eci_3[3,i];eci_4[1,i] eci_4[2,i] eci_4[3,i]]
satposes2 = [eci_1[1,i+1] eci_1[2,i+1] eci_1[3,i+1];eci_2[1,i+1] eci_2[2,i+1] eci_2[3,i+1];eci_3[1,i+1] eci_3[2,i+1] eci_3[3,i+1];eci_4[1,i+1] eci_4[2,i+1] eci_4[3,i+1]]
zenith_angles = zenith_angle(satposes, r0[1:3])
zenith_angles2 = zenith_angle(satposes2, r0[1:3])

println("satposes: ", satposes)

println("satposes2: ", satposes2)
    
zdot = zenith_angles - zenith_angles2

#check all angles to see if it is on the horizon 
    
inhorizon = all(x->x<70, zenith_angles)
    
#if inhorizon==false
#    continue #skip the current iteration
#end

#In km then to custom units
r1 = [eci_1[1,i], eci_1[2,i], eci_1[3,i]]*1e-3/distance_scale
r2 = [eci_2[1,i], eci_2[2,i], eci_2[3,i]]*1e-3/distance_scale
r3 = [eci_3[1,i], eci_3[2,i], eci_3[3,i]]*1e-3/distance_scale
r4 = [eci_4[1,i], eci_4[2,i], eci_4[3,i]]*1e-3/distance_scale
    
v1 = [eci_1[4,i], eci_1[5,i], eci_1[6,i]]*1e-3/velocity_scale
v2 = [eci_2[4,i], eci_2[5,i], eci_2[6,i]]*1e-3/velocity_scale
v3 = [eci_3[4,i], eci_3[5,i], eci_3[6,i]]*1e-3/velocity_scale
v4 = [eci_4[4,i], eci_4[5,i], eci_4[6,i]]*1e-3/velocity_scale

pose1 = [r1' v1']
pose2 = [r2' v2']
pose3 = [r3' v3']
pose4 = [r4' v4']
    
#at second frequency
pose5= [r1' v1']
pose6= [r2' v2']
pose7= [r3' v3']
pose8= [r4' v4']
    
time = [0.006, 0.006, 0.006, 0.008]/time_scale # assume a fixed time for the rotation matrix
    
#calculate the rdots. need to build the pose matrices
deltaf = measurment(r0_scaled, pose1, pose2, pose3, pose4, time)
    
pose1_rdot1 = [pose1 deltaf[1]]
pose2_rdot2 = [pose2 deltaf[2]]
pose3_rdot3 = [pose3 deltaf[3]]
pose4_rdot4 = [pose4 deltaf[4]]
    
all_sats_scaled = vcat(pose1_rdot1, pose2_rdot2, pose3_rdot3, pose4_rdot4)
    
println("this is deltaf: ", deltaf)

In [ ]:
initial_guess = [-0.963179168952113, -0.258771187758936, 0.07296286988883226, 0.3342312233885483]

In [ ]:
R = doppler_residual(initial_guess, all_sats_scaled, time)

In [ ]:
[-0.963179168952113, -0.258771187758936, 0.07296286988883226, 0.3342312233885483]